<img src="NotebookAddons/blackboard-banner.jpg" width="100%" />
<font face="Calibri">
<br>
<font size="7"> <b> GEOS 657: Microwave Remote Sensing<b> </font>

<font size="5"> <b>Lab 9: InSAR Time Series Analysis using GIAnT within Jupyter Notebooks<br>Part 1: Data Download & Preprocessing from a SARVIEWS Import <font color='rgba(200,0,0,0.2)'> -- [## Points] </font> </b> </font>

<br>
<font size="4"> <b> Franz J Meyer, Joshua J C Knicely, Alex Lewandowski, Rowan Biessel; University of Alaska Fairbanks</b> <br>
<img src="NotebookAddons/UAFLogo_A_647.png" width="170" align="right" /><font color='rgba(200,0,0,0.2)'> <b>Due Date: </b>NONE</font>
</font>

<font size="3"> This Lab is part of the UAF course <a href="https://radar.community.uaf.edu/" target="_blank">GEOS 657: Microwave Remote Sensing</a>. This lab is divided into 3 parts: 1) data download and preprocessing, 2) GIAnT time series, and 3) a simple Mogi source inversion. The primary goal of this lab is to demonstrate how to download the requisite data, specifically interferograms, and preprocess them for use with the Generic InSAR Analysis Toolbox (<a href="http://earthdef.caltech.edu/projects/giant/wiki" target="_blank">GIAnT</a>) in the framework of *Jupyter Notebooks*.<br>

<b>Our specific objectives for this lab are to:</b>

- Download data using ASF tools: 
    - From a SARVIEWS subscription. 
- Pre-process data: 
    - Subset (crop) the data to an Area of Interest (AOI). 
    - Verify the quality of the data.
    - Cull data selection based on a timeframe and orbital characteristics. 
    - Reproject interferograms to a uniform UTM zone. 
    - Use TRAIN to remove static atmospheric effects related to surface elevation.
</font>

<br>
<font face="Calibri">

<font size="5"> <b> Target Description </b> </font>

<font size="3"> In this lab, we will download interferograms covering a SARVIEWS area of interest.  </font>

<font size="4"> <font color='rgba(200,0,0,0.2)'> <b>THIS NOTEBOOK INCLUDES NO HOMEWORK ASSIGNMENTS.</b></font> <br>

Contact me at fjmeyer@alaska.edu should you run into any problems.
</font>

<font face='Calibri'><font size='5'><b>Overview</b></font>
<br>
<font size='3'><b>About TRAIN</b>
<br>
The tropospheric correction is one of the most significant challanges in InSAR. Without this correction, surface deformation signals can go completely unnoticed, or, perhaps worse, a false signal caused by the atmosphere can be taken as an accurate representation of surface deformation. This can often occur with volcanoes due to the characteristics of the atmosphere as well as the surface elevation (i.e., a taller point on the volcano means the InSAR signal passed through less atmosphere and will be affected differently from a point lower on the volcano). <br>We will use the Toolbox for Reducing Atmospheric InSAR Noise (TRAIN) today. The purpose of TRAIN is to add state of the art tropospheric correction methods to the InSAR processing chain. It can include corrections that are phase-based, using spectrometers, using weather models, and even data from balloon soundings. 
<br><br>
<b>Limitations</b>
<br>
The particular version we are using was created by the Alaska Satellite Facility. ASF took the original MATLAB code developed by David Bekaert created a Python2.7 wrapper (and more recently, a Python3.7 wrapper)for it. Currently, it only allows processing using MERRA2 data. Including other data types can be done relatively easily, though it does require modification of the existing python code. 
<br>
Each of the correction methods included in TRAIN is ideal for different locations and conditions. Spectrometers provide the best correction, but can only be used in cloud-free and daylight conditions. Phase-based and weather model correction methods capture regional signals well, but fail to capture turbulent tropospheric signals. 
<br><br>
More information about TRAIN, its capabilities, and its limitations can be found in <a href="http://www.sciencedirect.com/science/article/pii/S0034425715301231">Bekaert et al. [2015]</a>, at David Dekaert's <a href="http://davidbekaert.com/#links">webpage</a>, or at the <a href="https://github.com/asfadmin/hyp3-TRAIN" target="_blank">ASF Github</a>. 
<br><br>
<b>Steps to use TRAIN</b><br>

- System Setup
    - Import Python Packages
    - Set User Inputs
- Download and Preprocess Data
    - Access SARVIEWs Subscriptions
    - Download and unzip the Data
    - Project all Geotiffs to the Same UTM Zone
    - Mosaic Geotiffs with Partial Coverage
- Identify Area of Interest
- Subset (Crop) Data to Area of Interest
    - Subset the Data
    - Check that Subsetted Geotiffs have Pixels
    - Check the Dimensions of Subsetted Geotiffs
- Create Input Files and Code for TRAIN
    - Create <font face='Courier New'>parms_aps.txt</font> file
    - Create <font face='Courier New'>ifgday.mat</font> file
    - Convert Subsetted Tiffs to GCS Coordinates
    - Adjust file names
- Run TRAIN
    - Minor Set Up
    - Steps 0-3
    - Step 4
    - Comparison of Corrected and Uncorrected Unwrapped Phase
    - Convert back to the original coordinate system

<br><br>
When you use TRAIN, cite the creator's work using:<br>
Bekaert et al., RSE 2015, "Statistical comparison of InSAR tropospheric correction techniques." <br>&emsp;Open access: http://www.sciencedirect.com/science/article/pii/S0034425715301231
<!-- <br><br><b><i>DO WE NEED TO ALSO GIVE ASF A CITATION???</i></b> -->

<font face='Calibri'><font size='5'><b>0. Setup</b></font><br>
    <font size='3'>We will first import requisite Python libraries and modules, create or define an analysis directory, and define all of our user inputs. </font></font>

<font face='Calibri'>
<font size="4"> <b> 0.0 Import Python Packages and Enable Extensions</b></font>    <br>
    <font size='3'>First, we will import all needed libraries and modules. </font></font>

In [ ]:
import os
import shutil
import re
import glob
import json
import pickle
import pathlib
from datetime import date
from copy import copy
from urllib.parse import urlparse, parse_qs

import gdal
import pyproj
import numpy as np

import matplotlib.pyplot as plt
import matplotlib.animation
from matplotlib import animation
from matplotlib.widgets import RectangleSelector

from IPython.display import clear_output
from IPython.display import Markdown
plt.rcParams.update({'font.size': 12})

#from asf_notebook import path_exists
from asf_notebook import remove_nan_filled_tifs
from asf_notebook import select_parameter
from asf_notebook import EarthdataLogin
from asf_notebook import get_wget_cmd
from asf_notebook import asf_unzip
from asf_notebook import get_hyp3_subscriptions
from asf_notebook import gui_date_picker
#from asf_notebook import get_product_info
from asf_notebook import get_subscription_products_info
from asf_notebook import get_aquisition_date_from_product_name
from asf_notebook import get_products_dates_insar
from asf_notebook import get_slider_vals
from asf_notebook import input_path
from asf_notebook import handle_old_data

<font face='Calibri' size="4"> <b> 0.1 Define an Analysis Directory</b></font> 

In [ ]:
while True:
    sub_dir = input_path(
        f"\nPlease enter the name of a directory in which to store your data for this analysis.")
    if os.path.exists(sub_dir):
        contents = glob.glob(f'{sub_dir}/*')
        if len(contents) > 0:
            choice = handle_old_data(sub_dir, contents)
            if choice == 1:
                shutil.rmtree(sub_dir)
                os.mkdir(sub_dir)
                break
            if choice == 2:
                break
            else:
                clear_output()
                continue
        else:
            break
    else:
        os.mkdir(sub_dir)
        break
os.chdir(sub_dir)
home = os.getcwd()

<font face='Calibri' size="4"> <b> 0.2 Set Paths and TRAIN Parameters </b></font>   

<font face='Calibri'>
    <font size='3'><b>0.2.0 Create directories we will need later in the notebook</b><br></font>
</font>

In [ ]:
# Create a dictonary to hold values we wish to pickle and use in the Part 2 notebook
to_pickle = dict()

# Create the folder that will hold the full interferograms and their associated files. 
ingram_folder = 'ingrams'
pathlib.Path(ingram_folder).mkdir(parents=True, exist_ok=True)
to_pickle.update({'ingram_folder': ingram_folder})

# Create the folder in which we wish to store our interferogram subsets. 
subset_folder = 'ingram_subsets'
pathlib.Path(subset_folder).mkdir(parents=True, exist_ok=True)
to_pickle.update({'subset_folder': subset_folder})

            
# Create the folder in which we wish to store our converted interferogram subsets. 
# This is important later in the notebook when we convert our subsets from a 
# local geographic coordinate system to decimal degrees. 
corrected_folder = 'ingram_subsets_converted'
pathlib.Path(corrected_folder).mkdir(parents=True, exist_ok=True)
to_pickle.update({'corrected_folder': corrected_folder})


train_dir = '' # temporary blank directory
merra2_datapath = './MERRA2'
dem_path = os.path.join(train_dir,'myDEM.tif')

<font face='Calibri'>
    <font size='3'><b>0.2.1 Designate TRAIN Input Parameters</b><br></font>
</font>

In [ ]:
era_data_type = 'ECMWF'
ifgday_file = 'ifgday.mat'
lambda_meters = 0.055465763 
incidence_angle = 38.5/180*np.pi # This needs to be in radians. 
extra = 0.1

<font face='Calibri'><font size='4'><b>0.3 Earthdata Login</b><br></font>
<font size='3'>To download data from ASF, we need to provide our <a href="https://www.asf.alaska.edu/get-data/get-started/free-earthdata-account/" target="_blank">NASA Earth Data</a> username to the system. Setup an EarthData account if you do not yet have one. <font color='rgba(200,0,0,0.2)'><b>Note that EarthData's End User License Agreement (EULA) applies when accessing the Hyp3 API from this notebook. If you have not acknowleged the EULA in EarthData, you will need to navigate to <a href="https://earthdata.nasa.gov/" target="_blank">EarthData's home page</a> and complete that process.</b></font><br><br>
    For some data processing later, we will also need to add <b>NASA GESDISC DATA ARCHIVE</b> to our list of approved applications. This is needed to provide access to MERRA2 data files needed by TRAIN to perfrom the atmospheric correction. This can be done by going to your <a href="https://urs.earthdata.nasa.gov/profile" target="_blank">EarthData's profile page</a>, clicking <b>Applications</b> and selecting <b>Approved Applications</b> from the drop down menu, select <b>Approve More Applications</b> at the bottom left, search for <b>NASA GESDISC DATA ARCHIVE</b>, select it, and agree to the terms and conditions. Once that is complete, you will have access to the MERRA2 data and TRAIN will be able to automatically download whichever files it requires. 
<br><br>
<b>Login to Earthdata:</b> </font> 

In [ ]:
login = EarthdataLogin()

<hr>
<font face="Calibri">

<font size="5"> <b> 1. Download and Preprocess Data</b> </font>

<font size="3"> We will now access our subscriptions for download. 
    <br>
    1.0 and 1.1 demonstrate downloading from non-SARVIEWS and SARVIEWS subscriptions. 
<br><br>
<font size='4'><b>1.0 Access Non-SARVIEWS HyP3 Subscriptions</b><br></font>
Skip ahead to section 1.1 for SARVIEWS
<br><br>
<font size="3"><b>1.0.0 Enter a Group ID if downloading from a non-SARVIEWS group subscription:</b> </font>
<font>

In [ ]:
while True:
    group_id = input("Leave blank or enter a Group ID")
    if group_id == '':
        group_id = None
        break
    try:
        int(group_id)
    except ValueError:
        print("ValueError: Enter an integer.")
        pass
    else:
        clear_output()
        print(f"Group ID: {group_id}")
        break

<font face="Calibri" size="3"><b>1.0.1 Select a subscription</b> </font>
<font>

In [ ]:
subscriptions = get_hyp3_subscriptions(login)
if len(subscriptions) > 1:
    display(Markdown("<text style='color:red;'>Note: After selecting a subscription, you must select the next cell before hitting the 'Run' button or typing Shift/Enter.</text>"))
    display(Markdown("<text style='color:red;'>Otherwise, you will simply rerun this code cell.</text>"))
    print('\nSelect a Subscription:')
subscription_id = select_parameter('', subscriptions)
subscription_id

<font face="Calibri" size='3'><b>1.0.2 Save subscription choice</b></font>

In [ ]:
try:
    subscription_id = subscription_id.value.split(':')[0]
    print(subscription_id)
except AttributeError:
    pass
    print("AttributeError: 'str' object has no attribute 'value'")
    display(Markdown("<text style='color:red;'>Try re-running the previous code cell.</text>"))

<font face="Calibri" size='3'><b>1.0.3 Load a date selection widget containing the subscription's date range</b></font>

In [ ]:
display(Markdown("<text style='color:red;'>Note: After selecting a date range, you should select the next cell before hitting the 'Run' button or typing Shift/Enter.</text>"))
display(Markdown("<text style='color:red;'>Otherwise, you may simply rerun this code cell.</text>"))
print('\nSelect a Date Range:')
products_info = get_subscription_products_info(subscription_id, login, group_id=group_id)
dates = get_products_dates_insar(products_info)
date_picker = gui_date_picker(dates)
date_picker

<font face="Calibri" size='3'><b>1.0.4 Save the selected date range.</b></font>

In [ ]:
date_range = get_slider_vals(date_picker)
date_range[0] = date_range[0].date()
date_range[1] = date_range[1].date()
print(f"Date Range: {str(date_range[0])} to {str(date_range[1])}")

<font face="Calibri" size='3'><b>1.0.5 Create a list of download URLs for all products within our date range.</b></font>

In [ ]:
download_urls = []
for p in products_info:
    dt = get_aquisition_date_from_product_name(p)
    if dt >= date_range[0] and dt <= date_range[1]:
        download_urls.append(p['url'])
download_urls.sort()
print(f"There are {len(download_urls)} products to download.")

<font face='Calibri'><font size='4'><b>1.1 Access SARVIEWs Subscription</b><br></font>
<font size='3'>The below code is a demonstration of how to download data via a SARVIEWs subscription. The selected HyP3 product IDs are stored in the URL. <br>The first cell below acquires user credentials and creates a .netrc file in order to access Earthdata. The 2nd cell executes some javascript that loads this URL into the notebook's python kernel. Subsequently, this URL is parsed and the IDs are extracted. Using the supplied event ID, the following cell compares each of the IDs to all of the products in this event group and then returns a list of URLS from products whos id is included in the URL.  <font face='Courier New'>products_temp</font>. The user name and password are the corresponding Earthdata user login. </font></font>

<font face='Calibri' size='3'><b>1.1.0 Use Javascript to extract this notebook's URL </b></font>

In [ ]:

%%javascript 
var kernel = Jupyter.notebook.kernel; 
var command = ["notebookUrl = ",
               "'", window.location, "'" ].join('')
// alert(command)
kernel.execute(command)

<font face='Calibri' size='3'><b>1.1.1 Parse the URL and retrieve product IDs and Event</b></font>

In [ ]:
parsed_url = urlparse(notebookUrl)
params = parse_qs(parsed_url.query)
try:
    ids = params['ids'][0].split(',')
    event_id = params['event'][0]
    
    if len(ids) == 0:
        print('No products were found from the url')
    else:
        print(f'Found {len(ids)} product IDs in the URL to prepare: ')
        for product in ids:
            print(product)

    print(f'Using eventId: {event_id}')    
except:
    display(Markdown(f'<text style=color:red> ERROR: Missing Data</text>'))
    display(Markdown(f'<text style=color:red> Go to <a href="http://sarviews-test.s3-website-us-east-1.amazonaws.com/" target="_blank"> sarviews-hazards.alaska.edu </a> to find SARVIEWS data.</text>'))

<font face='Calibri' size='3'><b>1.1.2 Use the Hyp3 API to fetch product download URLs</b></font>

In [ ]:
# Get download URLs using the event ID 
groups = login.api.get_groups_public(id=event_id)
groupName = groups[0]['name']
print(f'Loading Products from {groupName}...')

# Load every page of products
event_prods = []
page = 1
emptyQuery = False
while not emptyQuery:
    new_prods =  login.api.get_products_public(group_id=event_id, page=(page - 1))
    event_prods += new_prods
    if len(new_prods) == 0:
        emptyQuery = True
    else:
        print(f'Loaded page {page} of products')
        page += 1
        
download_urls = []
for product in event_prods:
    if str(product['id']) in ids:
        download_urls.append(product['url'])
        
if (len(download_urls) == len(ids)):
    print('All IDs are accounted for with download URLS! Ready to download.')
else: 
    print(f'Only {len(download_urls)} products match those selected from the URL and are ready to download. ')
download_urls.sort()

<font face='Calibri' size='4'><b>1.3 Download and unzip the data. </b></font>
<br>
<font size='3' face='Calibri'>We should now have a list of the products we wish to download. The code below will download the requisite zip files, unpack them into our designated folder, and then delete any remaining zip files. Removing the zip files helps to reduce space usage.</font>



<font size='3'><b>1.3.0 Download the Products</b><br></font>

In [ ]:
print(f"There are {len(download_urls)} products to download.")
full_ingram_path = f"{home}/{ingram_folder}"
if os.path.exists(full_ingram_path):
    product_count = 1
    for url in download_urls:
        print(f"\nProduct Number {product_count} of {len(download_urls)}:")
        product_count += 1
        
        parsed = urlparse(url)
        file_name = os.path.basename(parsed.path) 
        # print(f'Filename: {file_name}')
        
        # if not already present, we need to download and unzip products
        product_folder = file_name.split('.zip')[0]
        if not os.path.exists(product_folder):
            print(
                f"\n{product_folder} is not present.\nDownloading from {url}")
            cmd = get_wget_cmd(url, login)
            !$cmd
            if os.path.exists(file_name):
                zippedProduct = f"{home}/{file_name}"
                zippedProduct = zippedProduct.split('\n')[0]
                print(f"zipped Product: {zippedProduct}")

                asf_unzip(full_ingram_path, zippedProduct)

                try:
                    os.remove(file_name)
                except OSError:
                    pass
                print(f"\nDone.")
            else:
                print('Download failed, does this HyP3 product still exist?')
        else:
            print(f"{product_folder} already exists.")

<font face="Calibri">
    <font size="5"> <b> 2. Prepare the Tiffs for Processing</b> </font>
</font>

<font face='Calibri'>
    <font size='4'><b>2.0 Gather Amplitude, Coherence, and Interferogram Paths</b></font><br><br>
    <font size='3'><b>2.0.0 Write functions to grab and print the path information for the amplitude, unwrapped phase, and the coherence files.</b></font>
</font>

In [ ]:
def get_tiff_paths(paths):
    tiff_paths = !ls $paths | sort -t_ -k5,5
    return tiff_paths

def print_tiff_paths(tiff_paths):
    print("Tiff paths:")
    for p in tiff_paths:
        print(f"{p}\n")

<font face='Calibri' size='3'><b>2.0.1 Call the functions we just wrote to gather the file path information.</b></font>

In [ ]:
paths_amp    = f"{ingram_folder}/*/*_amp.tif"
paths_ingram = f"{ingram_folder}/*/*_unw_phase.tif"
paths_cohr   = f"{ingram_folder}/*/*_corr.tif"
amp_paths    = get_tiff_paths(paths_amp)
ingram_paths = get_tiff_paths(paths_ingram)
cohr_paths    = get_tiff_paths(paths_cohr)
print(f"amp_path[0]: {amp_paths[0]}")
print(f"ingram_path[0]: {ingram_paths[0]}")
print(f"cohr_path[0]: {cohr_paths[0]}")

<font face='Calibri'><font size='4'><b>2.1 Orbit Direction</b><br></font>
<font size='3'><font face='Calibri'><font size='3'>We will acquire orbit information about our interferograms, retain only those that match our desired orbit direction, and then pickle the variable <font face='Courier New'>heading_avg</font> for use later in Part 2 of this lab. </font></font>

<font face='Calibri' size='3'><b>2.1.0 Write functions to filter product paths by orbit direction and file type</b>
</font>

In [ ]:
def get_interferogram_headings(metadata_paths: list) -> dict:
    headings = {}
    for path in metadata_paths:
        with open(path, 'r') as metadata:
            for line in metadata:
                t = line.split(':')
                if 'Heading' in t[0]:
                    headings.update({parse_interferogram_name(path): float(t[1])})
    return headings

def get_predominate_orbit_direction(headings: list) -> str:
    ascending = 0
    descending = 0
    for heading in headings:
        if abs(heading) < 90.0:
            ascending += 1
        else:
            descending += 1
    if ascending >= descending:
        return 'ascending'
    else:
        return 'descending'    

def parse_interferogram_name(path: str) -> str:
    regex = '[0-9]{8}T[0-9]{6}_[0-9]{8}T[0-9]{6}'
    return re.search(regex, path)[0]

def filter_by_orbit(paths, orbit, headings):
    pths = copy(paths)
    for path in paths:
        if orbit == 'ascending' and \
        headings[parse_interferogram_name(path)] >= 90.0:
            pths.remove(path)
        elif orbit == 'descending' and \
        headings[parse_interferogram_name(path)] < 90.0:
            pths.remove(path)
    return pths

<font face='Calibri' size='3'><b>2.1.1 Use heading directions from the product metadata to find the predominate orbit direction. 
    <br>
Filter the path lists to the data by the predominate orbit direction.</b></font>

In [ ]:
# Create a list of paths to interferogram metadata
metadata_paths = []
for path in amp_paths:
    metadata_paths.append(path.replace('_amp.tif', '.txt'))
    
# Create a dictionary of interferogram headings 
# and find the average heading
headings = get_interferogram_headings(metadata_paths)
heading_avg = np.mean(list(headings.values()))
print(f'Average Heading: {heading_avg}')

# Determine the predominate orbit direction
orbit = get_predominate_orbit_direction(list(headings.values())) 
print(f'Primary orbit: {orbit}')

# Gather lists of amp, ingram, and cohrs paths
amps = filter_by_orbit(amp_paths, orbit, headings) 
ingrams = filter_by_orbit(ingram_paths, orbit, headings)
cohrs = filter_by_orbit(cohr_paths, orbit, headings)

print(len(amps)) # if this is zero, then something is wrong.

<font face='Calibri'><font size='3'><b>2.1.2 Pickle <font face='Courier New'>heading_avg</font> for GIAnT</b><br></font>
<font size='3'><font face='Calibri'><font size='3'>We will need the variable average heading in the Part 2 notebook for this lab, when using GIAnT.</font></font>

In [ ]:
to_pickle.update({'heading_avg': heading_avg})

<font face='Calibri'><font size='4'><b>2.2 Project all data to the Same UTM Zone</b><br></font>
<font size='3'><font face='Calibri'><font size='3'>Some of the geotiffs may use different UTM zones. In the code below, we will identify the predominate UTM zone and reproject the rest into that zone. </font></font>

<font size='3' face='Calibri'><b>2.2.0: Create a list of all of the geotiff files.</b></font>

In [ ]:
tiff_paths = amps + ingrams + cohrs
# print_tiff_paths(tiff_paths)

<font face="Calibri" size="3"><b>2.2.1 Gather the UTM definition types and zones for each product and determine which is the most commonly used in the data stack</b></font>

In [ ]:
def get_utm_zones_types(tiff_paths):
    utm_zones = []
    utm_types = []
    print('Checking UTM Zones in the data stack ...\n')
    for i, path in enumerate(tiff_paths):
        info = (gdal.Info(path, options = ['-json']))
        info = (json.loads(info))['coordinateSystem']['wkt']
        zone = info.split('ID')[-1].split(',')[1][0:-2]
        utm_zones.append(zone)
        typ = info.split('ID')[-1].split('"')[1]
        utm_types.append(typ)
    return utm_zones, utm_types

utm_zones, utm_types = get_utm_zones_types(tiff_paths)
print(f"Unique UTM Zones: {list(set(utm_zones))}")
print(f"Unique UTM Types: {list(set(utm_types))}\n")

utm_unique, counts = np.unique(utm_zones, return_counts=True)
a = np.where(counts == np.max(counts))
predominate_utm = utm_unique[a][0]
print(f"Predominate UTM Zone: {predominate_utm}")
print(f"Number of UTM Zones:  {len(utm_unique)}")
to_pickle.update({'utm': predominate_utm})

<font face="Calibri" size="3"><b>2.2.2 Make a list of indicies in utm_zones that need to be reprojected.</b></font>

In [ ]:
reproject_indicies = [i for i, j in enumerate(utm_zones) if j != predominate_utm]
print('Reprojecting %4.1f files' %(len(reproject_indicies)))
print(reproject_indicies)

<font face="Calibri" size="3"><b>2.2.3 Call gdalwarp to reproject any geotiffs not in the predominant UTM zone.</b> These will be stored in a new file with a leading 'r' for identification. The originals are then deleted and the new geotiffs renamed. These new geotiffs are placed in an entirely new file as GDAL may overwrite parts of the original file before accessing them. </font>

In [ ]:
for k in reproject_indicies:
    folder, tiff_name = os.path.split(tiff_paths[k])
    cmd = (f"gdalwarp -overwrite {folder}/{tiff_name} {folder}/r{tiff_name}"
           f"-s_srs {utm_types[k]}:{utm_zones[k]} -t_srs EPSG:{predominate_utm}")
    print(f"Calling the command: {cmd}")
    !{cmd}
    rm_command = f"rm {tiff_paths[k].strip()}"
    print(f"Calling the command: {rm_command}")
    !{rm_command}
    # remove the leading 'r' from the new file. 
    print(f"Old : {os.path.join(folder,'r'+tiff_name)}")
    print(f"New : {os.path.join(folder,tiff_name)}")
    os.rename(os.path.join(folder,'r'+tiff_name),os.path.join(folder,tiff_name))

<font face="Calibri" size="3"><b>2.2.4 Double check that all of the 'tiff's now have the same UTM Zone and type and assign it to the 'utm' variable..</b></font>

In [ ]:
utm_zones, utm_types = get_utm_zones_types(tiff_paths)
zones = list(set(utm_zones))
print(f"Unique UTM Zones: {zones}")
print(f"Unique UTM Types: {list(set(utm_types))}")

if len(zones) == 1:
    utm = utm_zones[0][:]
    print(f"UTM Zone: {utm}")
else:
    print("After reprojection, there should be a single utm zone.")
    print(f"There are currently {len(zones)} zones.")

<font face='Calibri'><font size='4'><b>2.3 Mosaic Geotiffs with Partial Coverage</b><br></font>
    <font size='3'>Merge multiple frames of the same type and from the same date into a single geotiff. This code makes the assumption that any imagery taken on the same day are frames that do not overlap the same areas. For Sentinel1 imagery, this holds true; for other data sources, it may not. </font></font>

<font face='Calibri'><font size='3'><b>2.3.0 Get the paths for the files to be merged.</b></font>

In [ ]:
paths_full_amp  = f"{ingram_folder}/*/*_amp.tif"
paths_full_corr = f"{ingram_folder}/*/*_corr.tif"
paths_full_unw  = f"{ingram_folder}/*/*_unw_phase.tif"
amp_full_paths  = get_tiff_paths(paths_full_amp)
corr_full_paths = get_tiff_paths(paths_full_corr)
unw_full_paths  = get_tiff_paths(paths_full_unw)
print_tiff_paths(amp_full_paths+corr_full_paths+unw_full_paths)

<font face='Calibri'><font size='3'><b>2.3.1 Gather date info from filenames</b></font>

In [ ]:
def get_dates(paths):
    dates = []
    for p in paths:
        name = parse_interferogram_name(p)
        date1 = name.split("_")[0][0:8]
        date2 = name.split("_")[1][0:8]
        dates.append([date1,date2])
    dates.sort()
    return dates       

dates = get_dates(unw_full_paths)
for datepair in dates: print(datepair)

<font face="Calibri" size="3"><b>2.3.2 Create a list of groups of paths to products which share acquisition dates</b></font>

In [ ]:
dup_date_batches = []
dup_dates = []
for i in range(0,len(dates)-1):
    dte = dates[i]
    for j in range(i+1,len(dates)):
        if dates[j] == dte:
            dup_dates.append(dte)
print(f"dup_dates: {dup_dates}")

<font face="Calibri" size="3"><b>2.3.3 Write a function using gdal_merge to merge products with duplicate dates</b></font>

In [ ]:
def merge_dup_pairs(dups, paths):
    for date_pair in dups:
        matching = [s for s in paths if all(xs in s for xs in date_pair)]
        if len(matching) == 2:
            # create the output file name and merge the two files
            # with matching sets of dates. 
            path,file = os.path.split(matching[0])
            outputFile = os.path.join(path, 'MERGED'+file)
            cmd = f"gdal_merge.py -o {outputFile} {matching[0]} {matching[1]}"
            !{cmd}
            
            # The below code does some clean up. 
            # First, delete the unmerged original
            os.remove(matching[0])
            # Second, rename merged file to be that of the original file
            os.rename(outputFile,matching[0])
            # Third, remove matching[1] from the list of paths. 
            paths.remove(matching[1])
        else:
            print(f"Error: there is not a pair of matching entries.")
            print(f"Number of matches: {len(matching)}")
    return None 

<font face="Calibri" size="3"><b>2.3.4 Call merge_dup_pairs to merge amp, unw, and corr products with duplicated dates</b></font>

In [ ]:
merge_dup_pairs(dup_dates, amp_full_paths)
merge_dup_pairs(dup_dates, unw_full_paths)
merge_dup_pairs(dup_dates, corr_full_paths)
tiff_paths = amp_full_paths + unw_full_paths + corr_full_paths
print_tiff_paths(tiff_paths)

<font face='Calibri'>
    <font size='5'> <b> 3. Subset Area of Interest</b> </font>
    <br>
    <font size='3'> Here we identify our area of interest (AOI). Our AOI must contain all of the expected deformation and a surrounding region of little to no deformation. Following our selection of this region, we will subset our data to this region. This helps reduce computation time. </font>
    <br><br>
    <font size='4'><b>3.0 Identify AOI</b></font> <br><br>
    <font size='3'><b>3.0.0 Update the paths to the amplitude data after merging in the previous section</b></font> 
</font>

In [ ]:
amp_tiff_paths = get_tiff_paths(paths_full_amp)
# print_tiff_paths(amp_tiff_paths)

<font face='Calibri' size="3"> <b>3.0.1 Merge together one image from each area represented in the stack for display in the AOI selector:</b> </font> 

In [ ]:
to_merge = {}
for pth in amp_tiff_paths:
    info = (gdal.Info(pth, options = ['-json']))
    info = (json.loads(info))['wgs84Extent']['coordinates']
    
    coords = [info[0][0], info[0][3]]
    for i in range(0, 2):
        for j in range(0, 2):
            coords[i][j] = round(coords[i][j])
    str_coords = f"{str(coords[0])}{str(coords[1])}"
    if str_coords not in to_merge:
        to_merge.update({str_coords: pth})
merge_paths = ""
for pth in to_merge:
    merge_paths = f"{merge_paths} {to_merge[pth]}"  
print(merge_paths)

full_scene = f"{home}/full_scene.tif"
if os.path.exists(full_scene):
    os.remove(full_scene)
gdal_command = f"gdal_merge.py -o {full_scene} {merge_paths}"
!{gdal_command}

<font face="Calibri" size="3"><b>3.0.2 Create a VRT (virtual raster stack) from merged images and convert that into array:</b> </font>

In [ ]:
image_file = f"{home}/raster_stack.vrt"
!gdalbuildvrt -separate $image_file -overwrite $full_scene

img = gdal.Open(image_file)
rasterstack = img.ReadAsArray()

<font face="Calibri" size="3"><b>3.0.3 Write an AOI selector class:</b> </font>

In [ ]:
class AOI_Selector:
    def __init__(self, 
                 image,
                 fig_xsize=None, fig_ysize=None,
                 cmap=plt.cm.gist_gray,
                 vmin=None, vmax=None
                ):
        display(Markdown(f"<text style=color:blue><b>Area of Interest Selector Tips:\n</b></text>"))
        display(Markdown(f'<text style=color:blue>- This plot uses "matplotlib notebook", whereas the other plots in this notebook use "matplotlib inline".</text>'))
        display(Markdown(f'<text style=color:blue>-  If you run this cell out of sequence and the plot is not interactive, rerun the "%matplotlib notebook" code cell.</text>'))
        display(Markdown(f'<text style=color:blue>- Use the pan tool to pan with the left mouse button.</text>'))
        display(Markdown(f'<text style=color:blue>- Use the pan tool to zoom with the right mouse button.</text>'))
        display(Markdown(f'<text style=color:blue>- You can also zoom with a selection box using the zoom to rectangle tool.</text>'))
        display(Markdown(f'<text style=color:blue>- To turn off the pan or zoom to rectangle tool so you can select an AOI, click the selected tool button again.</text>'))
        
        display(Markdown(f'<text style=color:red><b>IMPORTANT!</b></text>'))
        display(Markdown(f'<text style=color:red>- Upon loading the AOI selector, the selection tool is already active.</text>'))
        display(Markdown(f'<text style=color:red>- Click, drag, and release the left mouse button to select an area.</text>'))
        display(Markdown(f'<text style=color:red>- The square tool icon in the menu is <b>NOT</b> the selection tool. It is the zoom tool.</text>'))
        display(Markdown(f'<text style=color:red>- If you select any tool, you must toggle it off before you can select an AOI</text>'))
        
        self.image = image
        self.x1 = None
        self.y1 = None
        self.x2 = None
        self.y2 = None
        if not vmin:
            self.vmin = np.nanpercentile(self.image, 1)
        else:
            self.vmin = vmin
        if not vmax:
            self.vmax=np.nanpercentile(self.image, 99)
        else:
            self.vmax = vmax
        if fig_xsize and fig_ysize:
            self.fig, self.current_ax = plt.subplots(figsize=(fig_xsize, fig_ysize)) 
        else:
            self.fig, self.current_ax = plt.subplots() 
        self.fig.suptitle('Area-Of-Interest Selector', fontsize=16)
        self.current_ax.imshow(self.image, cmap=plt.cm.gist_gray, vmin=self.vmin, vmax=self.vmax)


        def toggle_selector(self, event):
            print(' Key pressed.')
            if event.key in ['Q', 'q'] and toggle_selector.RS.active:
                print(' RectangleSelector deactivated.')
                toggle_selector.RS.set_active(False)
            if event.key in ['A', 'a'] and not toggle_selector.RS.active:
                print(' RectangleSelector activated.')
                toggle_selector.RS.set_active(True)
                
        toggle_selector.RS = RectangleSelector(self.current_ax, self.line_select_callback,
                                               drawtype='box', useblit=True,
                                               button=[1, 3],  # don't use middle button
                                               minspanx=5, minspany=5,
                                               spancoords='pixels',
                                               rectprops = dict(facecolor='red', edgecolor = 'yellow', 
                                                                alpha=0.3, fill=True),
                                               interactive=True)
        plt.connect('key_press_event', toggle_selector)

    def line_select_callback(self, eclick, erelease):
        'eclick and erelease are the press and release events'
        self.x1, self.y1 = eclick.xdata, eclick.ydata
        self.x2, self.y2 = erelease.xdata, erelease.ydata
        print("(%3.2f, %3.2f) --> (%3.2f, %3.2f)" % (self.x1, self.y1, self.x2, self.y2))
        print(" The button you used were: %s %s" % (eclick.button, erelease.button))

<font face="Calibri" size="3"><b>3.0.4 Create an AOI selector from your raster stack:</b> </font>

In [ ]:
%matplotlib notebook
fig_xsize = 7.5
fig_ysize = 7.5
aoi = AOI_Selector(rasterstack, fig_xsize, fig_ysize)

<font face="Calibri" size="3"><b>3.0.5 Gather and define projection details:</b> </font>

In [ ]:
geotrans = img.GetGeoTransform()
projlatlon = pyproj.Proj('EPSG:4326') # WGS84
projimg = pyproj.Proj(f'EPSG:{utm}')

<font face="Calibri" size="3"><b>3.0.6 Write a function to convert the pixel, line coordinates from the AOI selector into geographic coordinates in the stack's EPSG projection:</b> </font>

In [ ]:
def geolocation(x, y, geotrans,latlon=True):
    ref_x = geotrans[0]+x*geotrans[1]
    ref_y = geotrans[3]+y*geotrans[5]
    if latlon:
        ref_y, ref_x = pyproj.transform(projimg, projlatlon, ref_x, ref_y)
    return [ref_x, ref_y]

<font face="Calibri" size="3"><b>3.0.7 Call geolocation to gather the aoi_coords:</b> </font>

In [ ]:
try:
    aoi_coords = [geolocation(aoi.x1, aoi.y1, geotrans, latlon=False), geolocation(aoi.x2, aoi.y2, geotrans, latlon=False)]
    print(f"aoi_coords in EPSG {utm}: {aoi_coords}")
except TypeError:
    print('TypeError')
    display(Markdown(f'<text style=color:red>This error occurs if an AOI was not selected.</text>'))
    display(Markdown(f'<text style=color:red>Note that the square tool icon in the AOI selector menu is <b>NOT</b> the selection tool. It is the zoom tool.</text>'))
    display(Markdown(f'<text style=color:red>Read the tips above the AOI selector carefully.</text>'))

<font face='Calibri' size='4'><b> 3.1 Subset (Crop) Data to Area of Interest </b> </font>
<br>
<font face='Calibri' size='3'>We now subset our data to our AOI. We must do this for both the interferograms and the coherence files. In this lab, we will also subset the amplitude image files for later display purposes, though this is not necessary.</font>

In [ ]:
tiff_paths = amp_full_paths + unw_full_paths + corr_full_paths
print("Subsetting amplitude, coherence, and interferogram files.")
for tiff_path in tiff_paths:
    path,tiff = os.path.split(tiff_path)

    gdal_command = (f"gdal_translate -epo -eco -projwin {aoi_coords[0][0]} "
                    f"{aoi_coords[0][1]} {aoi_coords[1][0]} {aoi_coords[1][1]} "
                    f"-projwin_srs 'EPSG:{utm}' -co \"COMPRESS=DEFLATE\" "
                    f"-a_nodata 0 {tiff_path} {subset_folder}/{tiff} > /dev/null")

    # print(f"\nCalling the command: {gdal_command}") 
    !{gdal_command} # Call the GDAL command. 
print("Subsetting complete.")

<font size='3'><font face='Calibri'><font size='3'><b>NOTE:</b> It is possible GDAL returned an <b>error</b> for one or more of the interferogram, coherence, and amplitude files. This is because of our use of the "-epo" and "-eco" options. Those rasters which return an error are either partially or completely outside of our AOI. Including these empty files would cause errors in TRAIN and GIAnT. It is possible to include those files that only have partial extent within our AOI, but would require significantly more advanced processing which we exclude for simplicity. </font>

<font face='Calibri'><font size='4'><b>3.2 Perform some checks on the subset data</b></font>
<br>  
<font size='3'><b>3.2.0 Check that the Subsetted Tiffs have Pixels</b></font>
<br>
<font size='3'>Some of the subsetted geotiffs do not have pixels in our AOI despite the "-epo" and "-eco" options which should cause an error for all of these and skip them. Below, we will <b>check which geotiffs actually have pixels in our AOI and remove those that don't.</b> This can be done with a simple NaN search or by checking the band statistics of the file. </font>
</font>

In [ ]:
paths_subsets = f"{subset_folder}/*.tif"
subset_paths = get_tiff_paths(paths_subsets)
remove_nan_filled_tifs(subset_folder,subset_paths) 

<font face='Calibri' size='3'><b>3.2.1 Check the dimensions of the subset tiffs</b>
<br>
In some instances, the gdal_translate function may return subset imagery with slightly different extents. They may differ in size by a single pixel width in the x or y dimension. This is usually only a problem when multiple data sensors are used. Sensors will often have different pixel sizes and/or their pixel locations will be slightly offset from each other. Since all of our data comes from Sentinel-1, we probably don't need to worry but it is still good to double check.</font>


In [ ]:
subset_paths = get_tiff_paths(paths_subsets)
subset_paths.sort()
print(f"Number of subset tifs: {len(subset_paths)}")
for path in subset_paths:
    print(path)

<font face='Calibri'><font size='3'><b>3.2.2 Check that all subset images have the same dimensions</b></font>

In [ ]:
def get_pixels_lines(geotiff_paths: list) -> list:
    pixels = []
    lines = []
    for file in geotiff_paths:
        img = gdal.Open(file)
        pixels.append(img.RasterXSize)
        lines.append(img.RasterYSize)
    return {'pixels': set(pixels), 'lines': set(lines)}


dimensions = get_pixels_lines(subset_paths)
print(f"Dimensions: {dimensions}")
if len(dimensions['pixels']) > 1 or len(dimensions['lines']) > 1:
    print("Error: Not all subset images in stack have the same dimensions")
else:
    print("Check successful. All subset images in stack have the same dimensions.")

<font face='Calibri'>
<font size='5'> <b> 4. Create Input Files and Code for TRAIN </b> </font>
<br>
<font size='3'> In this section, we will use TRAIN to remove static atmospheric effects that can cause decoherence of the interferograms. This primarily corrects for effects caused by elevation differences between locations. <br>If we think of the propogating radar wave as a set of discrete rays, each ray will follow a different path. Those that reflect from elevated locations will pass through less of the atmosphere and therefore be less altered than those rays that reflect from locations at lower elevations. Without this correction, interferograms often produce exaggerated deformation. This is especially important in hazard monitoring of active volcanoes as a false alarm can be extremely costly and cause the general public to ignore future warnings. 
</font>
    
</font>

<font face='Calibri'>
    <font size='3'>Let's create the input files and modify the files as required by TRAIN. The necessary items and actions are listed below. <br>
        
- parms_aps.txt
    - List of parameters defining how TRAIN will run.
- ifgday.txt
    - Text file listing the primary and secondary date pairs. 
    - 2 column vector [primary secondary]
    - Format: YYYYMMDD
- Convert subsetted '.tif' Files to GCS Coordinates
    - TRAIN requires the input '.tif' files to have a global coordinate system. 
    - We will convert them to EPSG:4326. 
- Adjust File Names
    - Many SAR codes expect the input files to have a particular name format. 
    - For TRAIN, this is <font face='Courier New'>&lt;primary\_date&gt;\_&lt;secondary\_date&gt;\_&lt;unwrapped, amplitude, or coherence designation&gt;.tif</font>.
<br></font>
    </font>

<font face='Calibri'>
    <font size='4'> <b>4.0 Create parms_aps.txt file</b> </font>
    <br>
    <font size='3'>We will need to create the file 'parms_aps.txt'. TRAIN will read parameters from this file. We will first need to extract some information from the satellite metadata files. We'll start with getting the UTC time of the satellite pass over our study area. <br>
        
</font>
    </font>

<font face='Calibri'>
<font size='3'><b>4.0.0 Extract the UTC Time</b>
<br>
This information is contained in the file name.<br>
Alternatively, it can be found in the metadata file named <font face='Courier New'>$<$primary timestamp$>$_$<$secondary timestamp$>$.txt</font> that comes with the interferogram.
</font>

In [ ]:
def getUTC_sat(files):
    utc_hr = []
    utc_min = []
    utc_sec = []
    for file in files:
        vals = file.split('_')
        tstamp = vals[0][9:16]
        utc_hr.append(int(tstamp[0:2]))
        utc_min.append(int(tstamp[2:4]))
        utc_sec.append(int(tstamp[4:6]))    
    
    # UTC time as HH:MIN; we extract the median value and pad with up to 2 zeroes. 
    utc_sat = (f"{str(int(np.median(utc_hr))).zfill(2)}:"
               f"{str(int(np.median(utc_min))).zfill(2)}")
    # UTC time as an integer; Method from Tom Logan's prepGIAnT code
    # Can also be found inside <date>_<date>.txt file and hard coded/extracted
    c_l_utc = np.median(utc_hr)*3600 + np.median(utc_min)*60 + np.median(utc_sec) 
    return utc_sat, c_l_utc

unw_files = [f for f in os.listdir(subset_folder) if f.endswith('_unw_phase.tif')] 
print("Unwrapped Phase Tifs:")
for file in unw_files: 
    print(file)
    
utc_sat, c_l_utc = getUTC_sat(unw_files)
print(f"\nUTC in int form: {c_l_utc}")
print(f"UTC in human-readable form: {utc_sat}")

<font face='Calibri'>
    <font size='3'><b>4.0.1 Create parms_aps.txt file</b> </font>
    <br>
    <font size='3'>Make the parms_aps.txt file. This gives TRAIN information on how to process the data. </font>
<br><br>
<font color='red'>Note: Ignore the deprecation warnings generated by this code cell. This is happening under-the-hood in pyproj's geopandas calls and will go away with future updates. It is a known issue, not something we can control, and will not impact results.</font>
    </font>

In [ ]:
in_proj = pyproj.Proj(f'EPSG:{utm}')
out_proj = pyproj.Proj('EPSG:4326')
llx, lly = pyproj.transform(in_proj,out_proj,coords[0][0],coords[0][1])
urx, ury = pyproj.transform(in_proj,out_proj,coords[1][0],coords[1][1])
region_lats = abs(np.diff([lly,ury])[0]) + extra
region_lons = abs(np.diff([llx,urx])[0]) + extra

!mkdir -p {merra2_datapath} # create the directory

parms_aps_Template = '''
# Input parameters for TRAIN

crop_flag: n
date_origin: file
dem_null: -32768
DEM_origin: asf
DEM_file: {7}
era_data_type: {1}
ifgday_file: {2}
incidence_angle: {9}
lambda: {8}
look_angle: 21
meric_perc_coverage: 80
merra2_datapath: {4}
non_defo_flag: n
region_lat_range: {5}
region_lon_range: {6}
region_res: 0.008333000000000
save_folder_name: aps_estimation
small_baseline_flag: n
stamps_processed: n
UTC_sat: {3}

'''

with open(os.path.join(train_dir,'parms_aps.txt'), 'w') as fid:
    fid.write(parms_aps_Template.format(train_dir, era_data_type,
                                        ifgday_file, utc_sat,
                                        merra2_datapath, region_lats,
                                        region_lons, dem_path,
                                        lambda_meters, incidence_angle))

<font face='Calibri'><font size='3'>You may notice that the <font face='Courier New'>parms_aps.txt</font> file we created has very little geographic information; we've only included the width and height of the subsetted interferograms in decimal degrees (the variables <font face='Courier New'>region_lats</font> and <font face='Courier New'>region_lons</font>). This is because we will use one of our subsetted and converted tiffs as a geographic reference file. Otherwise, we would need to create a text file that contains the latitude and longitude of each pixel in the interferogram.</font>

<font face='Calibri'>
    <font size='4'><b>4.1 Create ifgday.mat file</b> </font>
    <br>
    <font size='3'>This gives TRAIN the primary and secondary dates of the interferograms as 2 column vectors. The dates must be separated by a single space.<br>
        
- Interferogram dates stored as a matrix with name ifgday and size [n_ifgs 2]. 
- Primary image is in the first and secondary is in the second column. 
- Specify dates as a numeric value in YYYYMMDD format.

    </font>
    </font>

In [ ]:
# Get all of the primary and secondary dates. 
primary_dates = []
secondary_dates = []
for file in unw_files:
    tstamps = file.split('_')
    primary_dates.append(tstamps[0][0:8])
    secondary_dates.append(tstamps[1][0:8])
# Sort the dates according to the primary dates. 
p_dates, s_dates = (list(t) for t in zip(*sorted(zip(primary_dates, secondary_dates))))

# write values to the 'ifgday_file'; make sure there is only 1 space between the dates.
with open( os.path.join(train_dir, ifgday_file), 'w') as ifg:
    for i, date in enumerate(p_dates):
        primary_date = date # pull out primary date
        secondary_date = s_dates[i] # pull out secondary date
        ifg.write(f'{primary_date} {secondary_date}\n') 

# print the contents of the file we just wrote
with open(os.path.join(train_dir,ifgday_file),'r') as ifg:
    ifg_contents = ifg.read()
print(ifg_contents) 

<font face='Calibri'>
    <font size='4'><b>4.2 Convert Subsetted '.tif' Files to GCS Coordinates</b> </font>
    <br>
    <font size='3'>The ASF version of TRAIN requries our subsets to be in GCS coordinates (i.e., for every pixel in the subset to be designated by latitude and longitude). Currently, they are in a UTM coordinate system, which gives pixel location in meters based on a local coordinate system. We now will reproject our subsets to GCS.
</font>

<font face='Calibri' size='3'><b>4.2.0 Define the projection to which we will be reprojecting the subsets. This is typically designated as EPSG:4326.</b></font>
 

In [ ]:
coord_TRAIN = '4326'

<font face='Calibri' size='3'><b>4.2.1 Create the converted subsets.</b></font>

In [ ]:
for file in unw_files:
    in_file = os.path.join(subset_folder, file)
    out_file = os.path.join(corrected_folder, file)
    cmd = f"gdalwarp -t_srs EPSG:{coord_TRAIN} {in_file} {out_file}"
    !{cmd}

<font face='Calibri'>
    <font size='4'><b>4.3 Adjust File Names</b></font>
    <font size='3'><br>TRAIN expects the file naming format, <font face='Courier New'>&lt;primary_date&gt;_&lt;secondary_date&gt;_&lt;unwrapped, amplitude, or coherence designation&gt;.tif</font>. We will adjust the files to match this name convention. The code below assumes that the files all come from Sentinel-1 and that every interferogram has a unique primary and secondary date pair. <br><b>This is not always true; some interferograms will have identical primary/secondary date pairs, but have been taken at different times.</b> This is a relatively rare occurrence, but it is good to keep in mind. To keep this exercise relatively simple, we assume each interferogram has a unique primary/secondary date pair.</font>
<br><br>
<b>4.3.0 Write a function to rename the files</b>
</font>

In [ ]:
def rename_files_for_train(data_dir, files): 
    for file in files:
        if "T" in file: # only change files needing to be renamed (containing a 'T') 
            old_name, old_ext = os.path.splitext(file)
            t_stamps = old_name.split('_')
            primary = t_stamps[0][0:8] 
            secondary = t_stamps[1][0:8]
            if "_unw" in file:
                new_name = f"{primary}_{secondary}_unw_phase{old_ext}"
            elif "_corr" in file:
                new_name = f"{primary}_{secondary}_corr{old_ext}"
            elif "_amp" in file:
                new_name = f"{primary}_{secondary}_amp{old_ext}"
            
            exists = os.path.isfile(os.path.join(data_dir, new_name))
            if exists:
                print(f"File already exists: {new_name}")
            else:
                os.rename(os.path.join(data_dir, file),
                          os.path.join(data_dir, new_name))
    print("Files renamed.")
    return

<font face='Calibri' size='3'><b>4.3.1 Rename the subset files and save a list of the new filenames.</b></font>

In [ ]:
file_ext = '.tif'
files = [f for f in os.listdir(subset_folder) if f.endswith(file_ext)] 
files.sort()

rename_files_for_train(subset_folder, files)
files_subset = [f for f in os.listdir(subset_folder)]

<font face='Calibri' size='3'><b>4.3.1 Rename the converted files and save a list of the new filenames.</b></font>

In [ ]:
files = [f for f in os.listdir(corrected_folder) if f.endswith(file_ext)] 
files.sort()

rename_files_for_train(corrected_folder, files)
files_converted = [f for f in os.listdir(corrected_folder)] 

<font face='Calibri'><font size='5'><b>5. Run TRAIN</b></font>
    <br>
    <font size='3'>We have now created all of the necessary files to run TRAIN. 

<font face='Calibri'><font size='4'><b>5.0 Minor Set Up</b></font>
    <br>
    <font size='3'>We have to set up some path information in order to run TRAIN. <i>Eventually, this will be modified so we don't have to include the full path to TRAIN.</i> Additionally, we show multiple ways in which to call TRAIN.</font></font>

In [ ]:
train_path = "/usr/local/TRAIN/src"
aps_weather_model_path = os.path.join(train_path,'aps_weather_model.py')
print(f"aps_weather_model.py path: {aps_weather_model_path}")
georef_path = os.path.join(home, corrected_folder, files_converted[0])
print(f"\nPath to a georeference file: {georef_path}")

In [ ]:
# Display some help information
!python $train_path/aps_weather_model.py -h

<font face='Calibri'><font size='4'><b>5.1 TRAIN Steps 0-3</b></font>
    <br>
    <font size='3'>Now we run steps 0 through 3. Step 4 requires a few extra actions. 
    <br><br>
        <b>Step 0: Identify weather data files to download.</b></font></font>

In [ ]:
netrc_path = '/home/jovyan/.netrc'
with open(netrc_path, 'w+') as netrc:
    netrc.write(f'machine urs.earthdata.nasa.gov login {login.username} password {login.password}')

!python $train_path/aps_weather_model.py -g {georef_path} 0 0

os.remove(netrc_path)

<font face='Calibri' size='3'><b>Step 1: Download weather data.</b>
<br>
This will download a series of '*.nc4' files for each primary.</font>

In [ ]:
# Delete any existing MERRA2 downloads to prevent old files from being used. 
try:
    shutil.rmtree(merra2_datapath)
except:
    pass

netrc_path = '/home/jovyan/.netrc'
with open(netrc_path, 'w+') as netrc:
    netrc.write(f'machine urs.earthdata.nasa.gov login {login.username} password {login.password}')

!python $train_path/aps_weather_model.py -g {georef_path} 1 1

os.remove(netrc_path)

<font face='Calibri' size='3'><b>Step 2: Calculate wet and hydrostatic zenith delays</b>
<br>
This will download a set of '*.xyz' files and use those 
to calculate the necessary delays. 
</font>

In [ ]:
netrc_path = '/home/jovyan/.netrc'
with open(netrc_path, 'w+') as netrc:
    netrc.write(f'machine urs.earthdata.nasa.gov login {login.username} password {login.password}')

!python $train_path/aps_weather_model.py -g {georef_path} 2 2

os.remove(netrc_path)

<font face='Calibri' size='3'><b>Step 3 - Calculate the SAR delays</b>
<br>
This produces *_*_{hydro_correction, wet_correction, and correction}.bin files
</font>

In [ ]:
netrc_path = '/home/jovyan/.netrc'
with open(netrc_path, 'w+') as netrc:
    netrc.write(f'machine urs.earthdata.nasa.gov login {login.username} password {login.password}')

!python $train_path/aps_weather_model.py -g {georef_path} 3 3

os.remove(netrc_path)

<font face='Calibri'><font size='3'>Step 3 may give a strange message: "No correction for &lt;insert date&gt;". If this occurs, it is most likely because TRAIN wasn't able to access the MERRA2 files due to missing permissions in your Earthdata user account. This can be done by going to your <a href="https://urs.earthdata.nasa.gov/profile" target="_blank">EarthData's profile page</a>, clicking <b>Applications</b> and selecting <b>Approved Applications</b> from the drop down menu. Select <b>Approve More Applications</b> at the bottom left, search for <b>NASA GESDISC DATA ARCHIVE</b>, select it, and agree to the terms and conditions. Once that is complete, restart the kernel in the kernel menu and rerun the notebook.</font></font>   

<font face='Calibri'><font size='4'><b>5.2 TRAIN Step 4</b></font>
<br>
<font size='3'>In step 4, we apply the correction to our <font face='Courier New'>&lt;\*\_\*\_unw\_phase.tif&gt;</font> files. 
<br><br>
For this step, we need to do 2 things first:
<ol>
<li>Move all of the '*.bin' files into the same directory as our converted geotiffs.</li>
<li>Move into the converted geotiffs directory.</li>
</ol>
<b>5.2.0 Create a list of our .bin files</b></font>     

In [ ]:
file_ext = '.bin'
files = [f for f in os.listdir('.') if f.endswith(file_ext)] 
files.sort()
print(len(files))

print(*files, sep="\n") 

<font face='Calibri' size='3'><b>5.2.1 Move the .bin files into the converted geotiffs directory and then move into that directory</b></font> 

In [ ]:
for file in files:
    try:
        shutil.move(file,os.path.join(corrected_folder,file))
    except FileNotFoundError:
        print(f"\nError, {file} does not exist.")
        print("Have you already moved your .bin files?")
        break

try:        
    os.chdir(f"{home}/{corrected_folder}")
except FileNotFoundError:
    raise

<font face='Calibri' size='3'><b>5.2.2 Step 4 - Subtract calculated delay from interferograms</b></font>

In [ ]:
netrc_path = '/home/jovyan/.netrc'
with open(netrc_path, 'w+') as netrc:
    netrc.write(f'machine urs.earthdata.nasa.gov login {login.username} password {login.password}')

!python $train_path/aps_weather_model.py -g {georef_path} 4 4

os.remove(netrc_path)

<font face='Calibri' size='3'><b>5.2.3 Return to the home directory for this analysis</b></font>

In [ ]:
try:
    os.chdir(home)
except FileNotFoundError:
    raise
    
print(f"Current Working Directory: {os.getcwd()}")

<font face='Calibri'><font size='3'>If we check our <font face='Courier New'>corrected_folder</font> directory, we will find new files with the naming convention <font face='Courier New'>&lt;\*\_\*\_unw\_phase\_corrected.tif&gt;</font>. The uncorrected files, <font face='Courier New'>&lt;\*\_\*\_unw\_phase.tif&gt;</font>, are now technically superfluous and can be deleted. However, we will keep these files for the purpose of comparing the corrected and uncorrected times series in Part 2.
</font></font>

<font face='Calibri'><font size='4'><b>5.3 Comparison of Corrected and Uncorrected Unwrapped Phase</b></font>
    <br>
    <font size='3'>We will make a quick and simple comparison between the corrected and uncorrected unwrapped phase geotiffs. This is meant to highlight the importance of these atmospheric corrections.
<br><br>
    <b>5.3.0 Gather the paths to the corrected and uncorrected unwrapped phase tifs</b></font>

In [ ]:
paths_cor = f"{home}/{corrected_folder}/*_unw_phase_corrected.tif"
paths_unc = f"{home}/{corrected_folder}/*_unw_phase.tif"
cor_paths = get_tiff_paths(paths_cor)
unc_paths = get_tiff_paths(paths_unc)

# Uncomment to view paths
# print_tiff_paths(cor_paths)
# print_tiff_paths(unc_paths)

<font face='Calibri' size='3'><b>5.3.1 Plot a corrected and uncorrected unwrapped phase tif for comparison</b></font>

In [ ]:
%matplotlib inline

corrected = gdal.Open(cor_paths[0])
uncorrected = gdal.Open(unc_paths[0])
im_c = corrected.GetRasterBand(1).ReadAsArray()
im_u = uncorrected.GetRasterBand(1).ReadAsArray()

fig = plt.figure(figsize=(18, 10))
ax1 = fig.add_subplot(221)
ax2 = fig.add_subplot(222)
ax1.imshow(im_c, cmap='gray')
ax2.imshow(im_u, cmap='gray')
plt.title('Corrected and Uncorrected Unwrapped Phase')

<font face='Calibri' size='3'><b>5.3.2 Plot a difference map of the two images</b></font>

In [ ]:
difference = np.subtract(im_c, im_u)
fig = plt.figure(figsize=(18, 10))
ax1 = fig.add_subplot(111)
fig_plot = ax1.imshow(difference, cmap='RdBu')
fig.colorbar(fig_plot, fraction=0.24, pad=0.02)
ax1.set(title='TRAIN Correction Difference Map [mm]')
plt.grid()

<font face='Calibri'><font size='3'>In looking at the Correction Difference Map, we can see that size of the correction correlates with surface elevation. Without this correction, a volcanologist would see the extra elevation difference as an indication of magma injection and possible eruptive activity.</font></font>

<font face='Calibri'><font size='4'><b>5.4 Convert back to original coordinate system</b></font>
<br>
<font size='3'>GIAnT requires the interferograms to be in a particular coordinate system. The original coordinate system is one of those accepted, so we will convert our interferograms back to that.
<br><br>
<b>5.4.0 Display our EPSGs</b>
</font>

In [ ]:
print(f"original coordiante system = EPSG:{utm}")
print(f"TRAIN coordinate system =    EPSG:{coord_TRAIN}")

<font face='Calibri' size='3'><b>5.4.1 Grab the paths to all the corrected tiffs</b></font>

In [ ]:
paths = f"{corrected_folder}/*.tif"
tiff_paths = get_tiff_paths(paths)

# Uncomment to view paths
print_tiff_paths(tiff_paths)

<font face='Calibri' size='3'><b>5.4.2 check that the current coordinate system of the files is different from the desired.</b></font>

In [ ]:
utm_zones, utm_types = get_utm_zones_types(tiff_paths)
print(f"Current UTM Types & Zones = EPSG:{list(set(utm_zones))}")
print(f"Expected current system   = EPSG:{coord_TRAIN}")
print(f"Desired coordinate system = EPSG:{utm}")

<font face='Calibri' size='3'><b>5.4.3 Reproject the corrected tiffs.</b></font>

In [ ]:
for file in tiff_paths:
    # Designate the output file and its path; ideally these are the same. 
    # GDAL can't do that (it'll overwrite data sometimes), so for each  
    # we create new file with the correct projection, delete the old file, 
    # and then rename the newly created file. 
    base_path ,filename = os.path.split(file)
    desig = 'TEMP_'
    outfile = os.path.join(base_path, f"{desig}{filename}")
    # create the convert command
    cmd = f"gdalwarp -t_srs EPSG:{utm} {file} {outfile}"
    !{cmd} # convert the file
    
    # delete the file in the EPSG:4326 coordinate system
    try:
        os.remove(file)
    except:
        pass
    
    # rename the file in the utm coordinate system to our original name of 'inFile'. 
    try:
        os.rename(outfile, file)
    except:
        pass

<font face='Calibri' size='3'><b>5.4.4 Confirm that the files are now in the correct projection</b></font>

In [ ]:
utm_zones, utm_types = get_utm_zones_types(tiff_paths)
print(f"Current UTM Types & Zones = EPSG:{list(set(utm_zones))}")
print(f"Expected current system   = EPSG:{utm}")
print(f"Desired coordinate system = EPSG:{utm}")

<font face='Calibri'><font size='4'><b>5.5 Take care of some final details</b></font>
    <br><br>
<font size='3'><b>5.5.0 Do another pixel check</b></font>
<br>
<font size='3'>Check the pixel sizes again, and then do the pixel correction if necessary.
</font></font>

In [ ]:
pixels_lines = get_pixels_lines(tiff_paths)
print(pixels_lines)
if len(pixels_lines['pixels']) == 1 and \
len(pixels_lines['lines']) == 1:
    print("All images in the stack have the same dimensions")
else:
    print("Error: Images with different dimensions exist in the stack")

<font face='Calibri' size='3'><b>5.5.1 Pickle the information we need to access in the Part 2 notebook</b></font>

In [ ]:
print(to_pickle)
filename = 'part1_pickle'
outfile = open(filename,'wb')

pickle.dump(to_pickle, outfile)
outfile.close()

<font face='Calibri'><font size='3'>
You have now corrected the interferograms for atmospheric conditions and can proceed to Part 2: GIAnT. 
</font></font>

<font face='Calibri' size='3'><b>Print the path to the analysis directory, which you can copy/paste into Part 2</b></font>

In [ ]:
print(home)

<font face="Calibri" size="2"> <i>GEOS 657 Microwave Remote Sensing - Version 2.0 - July 2020</i>
    <br>
        <i>Version Changes:</i>
    <br>
    <i>- Markdown editing</i>
    <br>
    <i>- General refactoring to unify code contributions and add robustness</i>
    
</font>